In [5]:
import pyspark

conf = pyspark.SparkConf()
conf.set("spark.jars.ivy","/tmp/.ivy")

sc = pyspark.SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 19:03:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

import pyspark.sql.functions as F

# from pyspark.sql.types import *

# Connect to remote spark session at port 8080
sess = (
    SparkSession.builder.appName("Streaming") # type: ignore
    .master("spark://spark:8080")
    .getOrCreate()
)
spark = SparkSession.builder.appName("Streaming").getOrCreate()

In [8]:
# Read data from the socket as a DataFrame.
socket_df = spark.readStream \
    .format("socket") \
    .option("host", "socket_stream") \
    .option("port", 9999) \
    .load()

# Parse JSON data using the defined schema
parsed_df = socket_df.selectExpr(
    "CAST(value AS STRING) as event_json",
    "get_json_object(value, '$.payload.payload.collection.slug') as collection_slug",
    "to_timestamp(get_json_object(value, '$.payload.sent_at')) as sent_at",
)
parsed_df.printSchema()
parsed_df.select("*")
# group by collection_slug and window of 1 hour every 30 seconds. Order by count descending
agg_df = parsed_df.groupBy(
    F.window("sent_at", "1 hour", "1 minute"), # Every 10 seconds, aggregate the data from the last 30 seconds
    "collection_slug"
).count()
agg_df.printSchema()


23/11/15 19:04:12 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


root
 |-- event_json: string (nullable = true)
 |-- collection_slug: string (nullable = true)
 |-- sent_at: timestamp (nullable = true)

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- collection_slug: string (nullable = true)
 |-- count: long (nullable = false)



In [9]:
# Start the Structured Streaming query
query = agg_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    # .trigger(processingTime="30 seconds")

query.start().awaitTermination()

23/11/15 19:04:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-eed1a7a9-9a6e-41f6-8e5f-fb5b01c584c1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/15 19:04:24 ERROR FileContext: org.apache.hadoop.security.KerberosAuthException: failure to login: javax.security.auth.login.LoginException: java.lang.NullPointerException: invalid null input: name
	at jdk.security.auth/com.sun.security.auth.UnixPrincipal.<init>(UnixPrincipal.java:71)
	at jdk.security.auth/com.sun.security.auth.module.UnixLoginModule.login(UnixLoginModule.java:134)
	at java.base/javax.security.auth.login.LoginContext.invoke(LoginContext.java:755)
	at java.base/javax.security.auth.login.LoginContext$4.run(LoginContext.java:679)
	at java.base/javax.security.auth.login.LoginContext$4.run(

Py4JJavaError: An error occurred while calling o59.start.
: java.lang.RuntimeException: org.apache.hadoop.security.KerberosAuthException: failure to login: javax.security.auth.login.LoginException: java.lang.NullPointerException: invalid null input: name
	at jdk.security.auth/com.sun.security.auth.UnixPrincipal.<init>(UnixPrincipal.java:71)
	at jdk.security.auth/com.sun.security.auth.module.UnixLoginModule.login(UnixLoginModule.java:134)
	at java.base/javax.security.auth.login.LoginContext.invoke(LoginContext.java:755)
	at java.base/javax.security.auth.login.LoginContext$4.run(LoginContext.java:679)
	at java.base/javax.security.auth.login.LoginContext$4.run(LoginContext.java:677)
	at java.base/java.security.AccessController.doPrivileged(AccessController.java:712)
	at java.base/javax.security.auth.login.LoginContext.invokePriv(LoginContext.java:677)
	at java.base/javax.security.auth.login.LoginContext.login(LoginContext.java:587)
	at org.apache.hadoop.security.UserGroupInformation$Hadoo

23/11/15 19:07:10 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-3f223707-d21e-4978-bc92-772e8201dab0. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-3f223707-d21e-4978-bc92-772e8201dab0
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.IndexedSeqOptimized.fore